# Load libraries and check compatibility

In [1]:
#load modules
import os
import sys
import requests
import tarfile
import zipfile
import shutil
import glob
from PIL import Image
import numpy as np
import tensorflow as tf

# need Python 3.6.x for tensorflow to work properly, check current version:
!python --version

# check tf version and current working directory
print("TensorFlow version: " + tf.__version__)
print(os.getcwd())

Python 3.6.6
TensorFlow version: 1.9.0
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS


# Download and unpack IICBU 2008 lymphoma dataset

In [2]:
#---------------------------------------------------------------------------------------------------------------------
#-- Many thanks to Magnus Erik Hvass Pedersen 2016, https://github.com/Hvass-Labs/TensorFlow-Tutorials, MIT license --
#---------------------------------------------------------------------------------------------------------------------

def maybe_download_and_extract(url, download_dir):
    """
    Download and extract the data if it doesn't already exist.
    Assumes the url is a tar-ball file.
    :param url:
        Internet URL for the tar-file to download.
        Example: "https://ome.grc.nia.nih.gov/iicbu2008/lymphoma.tar.gz"
    :param download_dir:
        Directory where the downloaded file is saved.
        Example: "iicbu2008/data"
    :return:
        Nothing.
    """

    # Filename for saving the file downloaded from the internet.
    # Use the filename from the URL and add it to the download_dir.
    filename  = url.split('/')[-1]
    file_path = os.path.join(download_dir, filename)
    
    # Check if the file already exists.
    # If it exists then we assume it has also been extracted,
    # otherwise we need to download and extract it now.
    if not os.path.exists(file_path):
        # Check if the download directory exists, otherwise create it.
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        # Check if the save directory exists, otherwise create it.
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
            
        r = requests.get(url,
                         stream=True, 
                         headers={'User-agent': 'Mozilla/5.0'} 
                         # the NIH website for some reason doesn't allow scraping, so we trick it into thinking we are a normal user with the header.
                         # the "requests.get" module was the only one I tried that worked well and didn't throw "403 forbidden."
                        )
        
        if r.status_code == 200:
            with open(file_path, 'wb') as f: # "wb" means "write binary"
                print ("Downloading %s, this will take a while (~1.5GB)" % filename)
                shutil.copyfileobj(r.raw, f)
        else:
            print("Response != 200, status not OK")
            
        print("Download finished. Extracting files.")

        if file_path.endswith(".zip"):
            # Unpack the zip-file.
            zipfile.ZipFile(file=file_path, mode="r").extractall(save_dir)
        elif file_path.endswith((".tar.gz", ".tgz")):
            # Unpack the tar-ball.
            tarfile.open(name=file_path, mode="r:gz").extractall(save_dir)
            
        print("Congratulations, %s was downloaded and extracted." % filename)
        
    else:
        print("Data has apparently already been downloaded and unpacked.")


#---------------------------------------------------------------------------------------------------------------


In [3]:
# Directory where you want to download the dataset.
download_dir = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data"
# Directory where you want to save the unpacked images.
save_dir = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images"
# URL for the dataset on the internet.
url = "https://ome.grc.nia.nih.gov/iicbu2008/lymphoma.tar.gz"

maybe_download_and_extract(url=url, 
                          download_dir=download_dir
                          )


Data has apparently already been downloaded and unpacked.


# Convert TIFF to JPEG
## Make sure all images are in folders with a name that matches the category
### If you used maybe_download_and_extract for the IICBU dataset, it's done already.
      E.g. \data\CLL\eji3019kd.tif
           \data\MCL\wlkj391.tif
 
 The file name doesn't matter, and we'll fix it in a minute anyway just to make things prettier.

In [4]:
# Convert all TIFF files to JPEG
current_path = save_dir
for root, dirs, files in os.walk(current_path, topdown=False):
    for name in files:
        print(os.path.join(root, name))
        #if os.path.splitext(os.path.join(root, name))[1].lower() == ".tiff":
        if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":
            if os.path.isfile(os.path.splitext(os.path.join(root, name))[0] + ".jpg"):
                print("A jpeg file already exists for %s" % name)
            # If a jpeg with the name does *NOT* exist, convert one from the tif.
            else:
                outputfile = os.path.splitext(os.path.join(root, name))[0] + ".jpg"
                try:
                    im = Image.open(os.path.join(root, name))
                    print ("Converting jpeg for %s" % name)
                    im.thumbnail(im.size)
                    im.save(outputfile, "JPEG", quality=100)
                except Exception as e: 
                  print(e)
                  

C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL1.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL100.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL101.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL102.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL103.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL104.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL105.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL106.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL107.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL108.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL109.jpg
C:\Users\hiltonc\Google Drive\cbeauhilton\DL MDS\iicbu\data\images\CLL\CLL11.jpg
C:\Users\hiltonc\Go

In [5]:

#----------------------------
#-- Clean up the directory --
#----------------------------

#-------------------------------------------------------------

# Move CLL TIFFs to new folder, leave JPEGs behind
CLL_src_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images\\CLL"
CLL_tiff_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\TIFFs\\CLL_TIFFs"

try:
  os.makedirs(CLL_tiff_fldr); ## create the destination folder
except:
  print ("Folder already exists")

for tiff_file in glob.glob(CLL_src_fldr+"\\*.tif"):
    shutil.move(tiff_file, CLL_tiff_fldr)

print("CLL TIFFs moved")
#-------------------------------------------------------------

# Move FL TIFFs to new folder, leave JPEGs behind
FL_src_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images\\FL"
FL_tiff_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\TIFFs\\FL_TIFFs"

try:
  os.makedirs(FL_tiff_fldr); ## create the destination folder
except:
  print ("Folder already exists")

for tiff_file in glob.glob(FL_src_fldr+"\\*.tif"):
    shutil.move(tiff_file, FL_tiff_fldr)
    
print("FL TIFFs moved")
#-------------------------------------------------------------

# Move MCL TIFFs to new folder, leave JPEGs behind
MCL_src_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images\\MCL"
MCL_tiff_fldr = "C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\TIFFs\\MCL_TIFFs"

try:
  os.makedirs(MCL_tiff_fldr); ## create the destination folder
except:
  print ("Folder already exists")

for tiff_file in glob.glob(MCL_src_fldr+"\\*.tif"):
    shutil.move(tiff_file, MCL_tiff_fldr)

print("MCL TIFFs moved")
#-------------------------------------------------------------

Folder already exists
CLL TIFFs moved
Folder already exists
FL TIFFs moved
Folder already exists
MCL TIFFs moved


In [6]:
#-------------------------------------------------------------
# Remove .ini files, if they exist
#-------------------------------------------------------------

folder_with_ini = save_dir
test = os.listdir(folder_with_ini)

for item in test:
    if item.endswith(".ini"):
        os.remove(os.path.join(folder_with_ini, item))
        
#-------------------------------------------------------------

folder_with_ini = CLL_src_fldr
test = os.listdir(folder_with_ini)

for item in test:
    if item.endswith(".ini"):
        os.remove(os.path.join(folder_with_ini, item))

#-------------------------------------------------------------

folder_with_ini = FL_src_fldr
test = os.listdir(folder_with_ini)

for item in test:
    if item.endswith(".ini"):
        os.remove(os.path.join(folder_with_ini, item))

#-------------------------------------------------------------

folder_with_ini = MCL_src_fldr
test = os.listdir(folder_with_ini)

for item in test:
    if item.endswith(".ini"):
        os.remove(os.path.join(folder_with_ini, item))

#-------------------------------------------------------------

print("Adios, ini")

Adios, ini


In [7]:
#------------------------------------------------------------------------------------

#rename CLL files
path = CLL_src_fldr
files = os.listdir(path)
i = 1

for file in files:
    os.rename(os.path.join(path, file), os.path.join(path, str('CLL')+str(i)+'.jpg'))
    i = i+1
    
#------------------------------------------------------------------------------------

# rename FL files
path = FL_src_fldr
files = os.listdir(path)
i = 1

for file in files:
    os.rename(os.path.join(path, file), os.path.join(path, str('FL')+str(i)+'.jpg'))
    i = i+1


#------------------------------------------------------------------------------------

# rename MCL files 
path = MCL_src_fldr
files = os.listdir(path)
i = 1

for file in files:
    os.rename(os.path.join(path, file), os.path.join(path, str('MCL')+str(i)+'.jpg'))
    i = i+1


#------------------------------------------------------------------------------------

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images\\CLL\\CLL100.jpg' -> 'C:\\Users\\hiltonc\\Google Drive\\cbeauhilton\\DL MDS\\iicbu\\data\\images\\CLL\\CLL2.jpg'

# Augmentation could go here

In [8]:
# The retrain.py program has some built-in augmentations,
# but if we wanted to, for example, do some custom augmentation
# such as patch-sampling, stain standardization, etc., it would be easiest here.
# Another advantage of doing augmentation before running the retrain.py program
# is that the augmentation within retrain.py prohibits using bottleneck pre-calculation,
# which extends training time hugely.

# Get the retrainer

In [9]:
# Get retraining program from GitHub and download into current working directory

url = "https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py"
directory = os.getcwd()
filename  = url.split('/')[-1]
file_path = os.path.join(directory, filename)

if not os.path.exists(file_path):
    r = requests.get(url)
    f = open(file_path,'wb')
    f.write(r.content)
else:
    print("Retrain.py file already exists")

Retrain.py file already exists


# Run retrain.py
#### Default retrainer uses InceptionV3, but can specify others.

#### Below are params and key defaults:
   
[--image_dir IMAGE_DIR] 
                  
[--output_graph OUTPUT_GRAPH]

[--intermediate_output_graphs_dir INTERMEDIATE_OUTPUT_GRAPHS_DIR]
                  
[--intermediate_store_frequency INTERMEDIATE_STORE_FREQUENCY]
                  
[--output_labels OUTPUT_LABELS]
                  
[--summaries_dir SUMMARIES_DIR]
                  
[--how_many_training_steps HOW_MANY_TRAINING_STEPS] default is 4000
                  
[--learning_rate LEARNING_RATE] default 0.01. Try 0.005.
                  
[--testing_percentage TESTING_PERCENTAGE] default is 10.
                  
[--validation_percentage VALIDATION_PERCENTAGE] default is 10.
                  
[--eval_step_interval EVAL_STEP_INTERVAL]
                  
[--train_batch_size TRAIN_BATCH_SIZE] default is 100.
                  
[--test_batch_size TEST_BATCH_SIZE]
                  
[--validation_batch_size VALIDATION_BATCH_SIZE] default is 100. "-1" uses entire validation set for each validation run, may increase training time.
                  
[--print_misclassified_test_images] default is False. 
                  
[--bottleneck_dir BOTTLENECK_DIR]
                  
[--final_tensor_name FINAL_TENSOR_NAME]
                  
[--flip_left_right] default is False. 
                  
[--random_crop RANDOM_CROP] default is 0, percentage. Try 5 and 10.
                  
[--random_scale RANDOM_SCALE] default is 0, percentage. Try 5 and 10.
                  
[--random_brightness RANDOM_BRIGHTNESS] default is 0, percentage. Try 5 and 10.
                  
[--tfhub_module TFHUB_MODULE]
                  
[--saved_model_dir SAVED_MODEL_DIR]





# To look at tensorboard:
Activate your tensorflow environment in a new command prompt then: 

tensorboard --logdir /tmp/retrain_logs

Then copy and paste the output URL into your browser (you may have to click through a firewall).

For me, this looks like:

C:\Users\hiltonc>activate tensorflow_3.6

(tensorflow_3.6) C:\Users\hiltonc>tensorboard --logdir /tmp/retrain_logs

TensorBoard 1.9.0 at http://14738542253ch:6006 (Press CTRL+C to quit)

In [ ]:
%run retrain.py \
--image_dir C:\\Users\\hiltonc\\"Google Drive"\\cbeauhilton\\"DL MDS"\\iicbu\\data\\images \
--how_many_training_steps 4000 \
--random_crop 0 \
--random_scale 0 \
--random_brightness 0 \
--flip_left_right False \
--learning_rate 0.01 \
--train_batch_size 100 \
--testing_percentage 10 \
--validation_percentage 10 \
--validation_batch_size 100 \
--print_misclassified_test_images True

INFO:tensorflow:Looking for images in 'CLL'
INFO:tensorflow:Looking for images in 'FL'
INFO:tensorflow:Looking for images in 'MCL'
INFO:tensorflow:Using C:\Users\hiltonc\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Conv2d_1a_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Conv2d_1a_3x3/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5b/Branch_1/Conv2d_0b_5x5/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_1/Conv2d_0b_1x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5c/Branch_1/Conv2d_0b_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_1/Conv_1_0c_5x5/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5c/Branch_1/Conv_1_0c_5x5/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_1/Conv_1_0c_5x5/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5c/Branch_1/Conv_1_0c_5x5/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5c/Branch_1/Conv_1

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5d/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5d/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_1/Conv2d_0b_5x5/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_5d/Branch_1/Conv2d_0b_5x5/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_5d/Branch_

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6a/Branch_1/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mi

...

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_0/Conv2d_0a_1x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_0/Conv2d_0a_1x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_0/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_1/Conv2d_0a_1x1/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_1/Conv2d_0a_1x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_6e/Branch_2/Conv2d_0d_7x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mi

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7a/Branch_1/Conv2d_0c_7x1/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_2/Conv2d_0a_1x1/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7b/Branch_2/Conv2d_0a_1x1/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_2/Conv2d_0b_3x3/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7b/Branch_2/Conv2d_0b_3x3/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_mean:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7b/Branch_2/Conv2d_0b_3x3/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7b/Branch_2/Conv2d

INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_1/Conv2d_0b_1x3/BatchNorm/moving_variance:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7c/Branch_1/Conv2d_0b_1x3/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_1/Conv2d_0b_1x3/weights:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7c/Branch_1/Conv2d_0b_1x3/weights
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_1/Conv2d_0c_3x1/BatchNorm/beta:0 from checkpoint b'C:\\Users\\hiltonc\\AppData\\Local\\Temp\\tfhub_modules\\11d9faf945d073033780fd924b2b09ff42155763\\variables\\variables' with InceptionV3/Mixed_7c/Branch_1/Conv2d_0c_3x1/BatchNorm/beta
INFO:tensorflow:Initialize variable module/InceptionV3/Mixed_7c/Branch_